In [6]:
import pandas as pd

orders = pd.read_csv('./data/orders.csv')
orders['Order Date'] = pd.to_datetime(orders['Order Date'])
furniture = pd.Series(orders[orders.Category == 'Furniture'].groupby('Order Date').agg('sum')['Quantity']).asfreq("W").fillna(0)
office = pd.Series(orders[orders.Category == 'Office Supplies'].groupby('Order Date').agg('sum')['Quantity']).asfreq("W").fillna(0).rename({"Quantity": "Office"})
technology = pd.Series(orders[orders.Category == 'Technology'].groupby('Order Date').agg('sum')['Quantity'])
technology['2014-01-01'] = 0
technology = technology.asfreq("W").fillna(0).rename({"Quantity": "Technology"})

/var/folders/3q/37b0tth55xxdfn82hcm7ylp80000gn/T/ipykernel_47742/625248955.py:4: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  orders['Order Date'] = pd.to_datetime(orders['Order Date'])


In [7]:
furniture.rename({"Quantity": "Furniture"}, inplace=True)
data = pd.concat([office, furniture, technology], axis=1)
data.columns = ['Office', "Furniture", "Technology"]

In [8]:
data = data.iloc[:205]
data.shape

(205, 3)

In [13]:
import numpy as np
import pandas as pd
from sktime.forecasting.var import VAR
from sktime.forecasting.model_selection import (
    ForecastingGridSearchCV,
    SlidingWindowSplitter
)
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error

# Ensure the index is a datetime index
data.index = pd.date_range(start="2000-01-01", periods=len(data), freq="D")

# Define 1-step ahead forecasting horizon
fh = [1]

# Define Sliding Window Cross-Validation
cv = SlidingWindowSplitter(
    window_length=50,  # Train on past 50 observations
    fh=fh,
    step_length=1  # Move the window 1 step forward
)

# Initialize VAR model
forecaster = VAR()

# Define hyperparameter grid
param_grid = {
    "maxlags": list(range(15))  # Lag order
}

# Grid Search with MAPE as the scoring metric
grid_searcher = ForecastingGridSearchCV(
    forecaster=forecaster,
    param_grid=param_grid,
    cv=cv,
    scoring=mean_absolute_percentage_error
)

# Fit the model
grid_searcher.fit(data)
print("Best Parameters:", grid_searcher.best_params_)

# Make 1-step ahead prediction using the best model
best_forecaster = grid_searcher.best_forecaster_
y_pred = best_forecaster.predict(fh)
print(y_pred)


/Users/grigorychaykovsky/retail_forecasts/.venv/lib/python3.12/site-packages/sktime/utils/parallel.py:92: FitFailedWarning: 
                In evaluate, fitting of forecaster VAR failed,
                you can set error_score='raise' in evaluate to see
                the exception message.
                Fit failed for the 0-th data split, on training data y_train with
                cutoff <NA>, and len(y_train)=50.
                The score will be set to nan.
                Failed forecaster with parameters: VAR(maxlags=0).
                
  ret = [fun(x, meta=meta) for x in iter]
/Users/grigorychaykovsky/retail_forecasts/.venv/lib/python3.12/site-packages/sktime/utils/parallel.py:92: FitFailedWarning: 
                In evaluate, fitting of forecaster VAR failed,
                you can set error_score='raise' in evaluate to see
                the exception message.
                Fit failed for the 1-th data split, on training data y_train with
                cutoff <NA

Best Parameters: {'maxlags': 7}
                Office  Furniture  Technology
2000-07-24  198.699155  58.569227   62.522975


In [14]:
grid_searcher.best_params_

{'maxlags': 7}

In [15]:
grid_searcher.best_forecaster_._fitted_forecaster.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sun, 02, Mar, 2025
Time:                     20:36:13
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                    19.0104
Nobs:                     198.000    HQIC:                   18.3580
Log likelihood:          -2550.37    FPE:                6.04372e+07
AIC:                      17.9144    Det(Omega_mle):     4.40587e+07
--------------------------------------------------------------------
Results for equation Office
                   coefficient       std. error           t-stat            prob
--------------------------------------------------------------------------------
const                18.648877         8.669614            2.151           0.031
L1.Office             0.270562         0.096915            2.792           0.005
L1.Furniture          0.038766         0.233169            

In [16]:
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError
from sktime.forecasting.model_evaluation import evaluate

mape = MeanAbsolutePercentageError(symmetric=False)

cv_results = evaluate(
    forecaster=grid_searcher.best_forecaster_,  
    y=data,
    cv=cv,
    strategy="refit",
    scoring=mape
)

In [27]:
import matplotlib.pyplot as plt
cv_results.test_MeanAbsolutePercentageError[cv_results.test_MeanAbsolutePercentageError < 100].mean()

0.8658879947147138

Как видим, в большинстве случаев модель дает относительно разумные предсказания, но иногда, из-за близости исходных значений к нулю, MAPE принимает экстремально большие значения, которые не имеет смысла принимать по внимание. Так или иначе, сравнивать с другими поделям нужно будем по средней абсолютной ошибке или по средней квадратичной ошибке.

# Triple ARIMA

In [ ]:
from sktime.forecasting.arima import ARIMA
from sktime.forecasting.model_selection import SlidingWindowSplitter, ForecastingGridSearchCV
from sktime.forecasting.base import ForecastingHorizon
from numpy.typing import ArrayLike

def grid_searcher_sliding_arima(
        data:ArrayLike, 
        max_p:int=10,
        max_q:int=10,
        max_d:int=1
    ) -> ForecastingGridSearchCV:
    fh = ForecastingHorizon(1)

    cv = SlidingWindowSplitter(start_with_window=True, fh=fh, step_length=10, window_length=50)
    forecaster = ARIMA(enforce_invertibility=False, enforce_stationarity=False, suppress_warnings=True)
    arima_orders = []
    start_p, start_q, start_d = 1, 1, 0
    for p in range(start_p, max_p+1):
        for q in range(start_q, max_q+1):
            for d in range(start_d, max_d + 1):
                if set([p, q]) != set([6, 7]):
                    arima_orders.append((p, d, q))
    param_grid = {"order": arima_orders}

    grid_searcher = ForecastingGridSearchCV(
        forecaster=forecaster,
        cv=cv,
        param_grid=param_grid,
        error_score='raise',
        backend="multiprocessing"
    )
    data.fillna(0, inplace=True)
    grid_searcher.fit(data)
    return grid_searcher

In [32]:
tech_gs = grid_searcher_sliding_arima(
    data=technology,
    max_p=10,
    max_q=10,
    max_d=1
)
office_gs = grid_searcher_sliding_arima(
    data=office,
    max_p=10,
    max_q=10,
    max_d=1
)
furniture_gs = grid_searcher_sliding_arima(
    data=furniture,
    max_p=10,
    max_q=10,
    max_d=1
)

In [36]:
print(
    f"\
    {tech_gs.cv_results_.mean_test_MeanAbsolutePercentageError.mean() = },\n\
    {office_gs.cv_results_.mean_test_MeanAbsolutePercentageError.mean() = },\n\
    {furniture_gs.cv_results_.mean_test_MeanAbsolutePercentageError.mean() = }\n\
    "
)

    tech_gs.cv_results_.mean_test_MeanAbsolutePercentageError.mean() = 0.5230651681578081,
    office_gs.cv_results_.mean_test_MeanAbsolutePercentageError.mean() = 0.404896504077993,
    furniture_gs.cv_results_.mean_test_MeanAbsolutePercentageError.mean() = 0.887560828360154
    


Получилось даже лучше, чем в случае векторной авторегрессии